In [11]:
import tensorflow as tf
import tensorflow.contrib
import tensorflow.contrib.rnn
import numpy as np
import datetime
import time
import os, shutil

In [2]:
def genObs(length):
    return np.random.choice(range(3), size=length)

#very simple -- gru-rnn with 5 neurons cracks it
#def genTarget(x):
#    return ''.join([str(z) for z in x]).find('012')

def genTarget(x):
    y0 = ''.join([str(z) for z in x])
    return y0.count(y0[:2])

def genSample(num, length=20):
    x = [genObs(length=length) for _ in range(num)]
    y = [genTarget(t) for t in x]
    return np.array(x), np.array(y)

def randomBatch(tensorTuple, batchSize=64):
    ids = np.random.choice(range(tensorTuple[0].shape[0]), batchSize)
    return (x[ids,] for x in tensorTuple)

In [3]:
train_x, train_y = genSample(20000)
valid_x, valid_y = genSample(2000)
valid_x[:10], valid_y[:10]

(array([[2, 0, 0, 2, 1, 0, 0, 0, 2, 2, 2, 0, 1, 1, 0, 0, 1, 0, 1, 2],
        [0, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 2, 0, 0, 0, 0, 2, 0, 1, 1],
        [1, 2, 0, 1, 0, 1, 2, 0, 1, 2, 0, 0, 1, 1, 2, 0, 1, 2, 2, 0],
        [0, 0, 0, 2, 0, 1, 1, 1, 0, 1, 2, 0, 1, 2, 2, 1, 2, 0, 0, 2],
        [0, 2, 0, 0, 0, 0, 2, 2, 2, 1, 0, 0, 0, 1, 2, 2, 1, 0, 2, 2],
        [2, 0, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 0, 2, 1, 0, 1, 2, 2, 2],
        [0, 2, 2, 1, 1, 1, 0, 0, 0, 2, 2, 0, 1, 0, 1, 2, 2, 0, 1, 2],
        [2, 0, 1, 0, 0, 0, 0, 2, 2, 1, 2, 0, 1, 0, 0, 2, 0, 2, 0, 1],
        [1, 2, 1, 2, 1, 1, 2, 2, 2, 2, 0, 2, 0, 1, 0, 2, 0, 0, 2, 0],
        [0, 1, 1, 0, 2, 0, 0, 1, 1, 0, 1, 2, 1, 0, 1, 0, 1, 0, 1, 2]]),
 array([2, 2, 5, 2, 3, 2, 2, 4, 3, 6]))

In [18]:
SEQ_LEN = 20
RNN_SIZE = [10]

InnerCell = lambda n: tf.nn.rnn_cell.GRUCell(num_units=n, activation=tf.nn.elu)
#InnerCell = lambda n: tf.contrib.rnn.GRUCell(num_units=n, activation=tf.nn.elu)
#InnerCell = lambda n: tf.nn.rnn_cell.LSTMCell(num_units=n, activation=tf.nn.relu, use_peepholes=True)

tf.reset_default_graph()

tfi_x = tf.placeholder(shape=(None, SEQ_LEN), dtype=tf.float32)
tfi_y = tf.placeholder(shape=(None), dtype=tf.float32)

tfX = tf.reshape(tfi_x, shape=(tf.shape(tfi_x)[0], tf.shape(tfi_x)[1], 1))
tfY = tf.reshape(tfi_y, shape=(tf.shape(tfi_y)[0], 1))


rnnCell = tf.nn.rnn_cell.MultiRNNCell([InnerCell(s) for s in RNN_SIZE])
#rnnCell = tf.contrib.rnn.MultiRNNCell([InnerCell(s) for s in RNN_SIZE])
#rnnCell = tf.nn.rnn_cell.GRUCell(RNN_SIZE[0])

_, tfO = tf.nn.dynamic_rnn(rnnCell, inputs=tfX, dtype=tf.float32)

tfOut = tf.layers.dense(tfO[-1], 1)

tfLoss = tf.sqrt(tf.reduce_mean(tf.square(tfY - tfOut)))
tfTrain = tf.train.AdamOptimizer(1e-3).minimize(tfLoss)

#tfOutP = tf.nn.softmax(tfOut)
tfOutR = tf.cast(tf.round(tfOut),dtype=tf.int64)

tfAccuracy = tf.reduce_mean(tf.cast(tf.equal(tf.cast(tfY, dtype=tf.int64), tf.cast(tf.round(tfOut),dtype=tf.int64)), dtype=tf.float32))

tfsumRMSE = tf.summary.scalar('RMSE', tfLoss)
tfsumAccuracy = tf.summary.scalar('Accuracy', 1-tfAccuracy)
tfsumAll = tf.summary.merge([tfsumRMSE, tfsumAccuracy])

merged_summary = tf.summary.merge_all()

print('Graph creation complete')

Graph creation complete


In [ ]:
dt_now = datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S")
tffw = tf.summary.FileWriter('D:/Jupyter/Logs/00_SeqRNN-{0}'.format(dt_now), tf.get_default_graph())

batch_size = 1000
num_steps  = 30
num_epochs = 5000

fmtstr = 'Epoch {0} ({1:1.3} sec): loss changed from {2:1.3} to {3:1.3}\t\tVL:{4:1.3f}\t\tAC:{5:1.3f}'
valid_batch = {tfi_x: valid_x, tfi_y: valid_y}
with tf.Session() as tfs:
    tfs.run(tf.global_variables_initializer())
    for i in range(num_epochs):
        mini_x, mini_y = randomBatch((train_x, train_y), batchSize=batch_size)
        train_batch = {tfi_x:mini_x, tfi_y:mini_y}
        l0 = tfLoss.eval(feed_dict=train_batch)
        t0 = time.perf_counter()
        for j in range(num_steps):
            tfTrain.run(feed_dict=train_batch)
        t1 = time.perf_counter()
        l1 = tfLoss.eval(feed_dict=train_batch)
        lv = tfLoss.eval(feed_dict=valid_batch)
        ac = tfAccuracy.eval(feed_dict=valid_batch)
        summary = tfsumAll.eval(feed_dict=valid_batch)
        tffw.add_summary(summary, i)
        if i%10 == 0 or i == num_epochs - 1:
            print(fmtstr.format(i,t1-t0,l0,l1,lv,ac))
    valid_r = tfs.run(tfOutR, feed_dict=valid_batch)

Epoch 0 (0.463 sec): loss changed from 2.81 to 2.09		VL:2.073		AC:0.135
Epoch 10 (0.452 sec): loss changed from 1.1 to 1.1		VL:1.141		AC:0.312
Epoch 20 (0.451 sec): loss changed from 1.12 to 1.12		VL:1.117		AC:0.320
Epoch 30 (0.446 sec): loss changed from 1.12 to 1.12		VL:1.113		AC:0.322
Epoch 40 (0.45 sec): loss changed from 1.18 to 1.17		VL:1.104		AC:0.322
Epoch 50 (0.426 sec): loss changed from 1.1 to 1.08		VL:1.061		AC:0.336
Epoch 60 (0.426 sec): loss changed from 1.08 to 1.06		VL:1.016		AC:0.363
Epoch 70 (0.462 sec): loss changed from 1.02 to 0.991		VL:0.996		AC:0.379
Epoch 80 (0.459 sec): loss changed from 0.956 to 0.936		VL:0.977		AC:0.377
Epoch 90 (0.449 sec): loss changed from 0.938 to 0.92		VL:0.964		AC:0.387
Epoch 100 (0.474 sec): loss changed from 0.919 to 0.909		VL:0.949		AC:0.399
Epoch 110 (0.442 sec): loss changed from 0.954 to 0.932		VL:0.938		AC:0.388
Epoch 120 (0.426 sec): loss changed from 0.914 to 0.894		VL:0.926		AC:0.398
Epoch 130 (0.426 sec): loss changed from 0.

Epoch 1090 (0.417 sec): loss changed from 0.251 to 0.237		VL:0.267		AC:0.932
Epoch 1100 (0.451 sec): loss changed from 0.259 to 0.248		VL:0.262		AC:0.937
Epoch 1110 (0.482 sec): loss changed from 0.257 to 0.243		VL:0.262		AC:0.937
Epoch 1120 (0.444 sec): loss changed from 0.255 to 0.242		VL:0.259		AC:0.938
Epoch 1130 (0.505 sec): loss changed from 0.247 to 0.231		VL:0.258		AC:0.939
Epoch 1140 (0.437 sec): loss changed from 0.255 to 0.238		VL:0.256		AC:0.938
Epoch 1150 (0.46 sec): loss changed from 0.243 to 0.227		VL:0.255		AC:0.946
Epoch 1160 (0.466 sec): loss changed from 0.248 to 0.23		VL:0.251		AC:0.944
Epoch 1170 (0.504 sec): loss changed from 0.245 to 0.228		VL:0.255		AC:0.939
Epoch 1180 (0.471 sec): loss changed from 0.264 to 0.245		VL:0.251		AC:0.941
Epoch 1190 (0.467 sec): loss changed from 0.247 to 0.232		VL:0.249		AC:0.947
Epoch 1200 (0.511 sec): loss changed from 0.24 to 0.225		VL:0.247		AC:0.944
Epoch 1210 (0.447 sec): loss changed from 0.253 to 0.234		VL:0.247		AC:0.947
Ep

Epoch 2160 (0.435 sec): loss changed from 0.162 to 0.146		VL:0.166		AC:0.988
Epoch 2170 (0.435 sec): loss changed from 0.169 to 0.154		VL:0.169		AC:0.990
Epoch 2180 (0.466 sec): loss changed from 0.165 to 0.149		VL:0.164		AC:0.989
Epoch 2190 (0.439 sec): loss changed from 0.156 to 0.143		VL:0.166		AC:0.988
Epoch 2200 (0.419 sec): loss changed from 0.162 to 0.149		VL:0.160		AC:0.992
Epoch 2210 (0.487 sec): loss changed from 0.157 to 0.146		VL:0.166		AC:0.988
Epoch 2220 (0.436 sec): loss changed from 0.155 to 0.145		VL:0.163		AC:0.990
Epoch 2230 (0.47 sec): loss changed from 0.154 to 0.142		VL:0.167		AC:0.988
Epoch 2240 (0.498 sec): loss changed from 0.154 to 0.141		VL:0.163		AC:0.990
Epoch 2250 (0.455 sec): loss changed from 0.15 to 0.139		VL:0.159		AC:0.992
Epoch 2260 (0.455 sec): loss changed from 0.152 to 0.142		VL:0.159		AC:0.992
Epoch 2270 (0.429 sec): loss changed from 0.158 to 0.144		VL:0.162		AC:0.992
Epoch 2280 (0.423 sec): loss changed from 0.161 to 0.15		VL:0.162		AC:0.989
Ep

Epoch 3230 (0.451 sec): loss changed from 0.134 to 0.121		VL:0.142		AC:0.994
Epoch 3240 (0.452 sec): loss changed from 0.129 to 0.121		VL:0.143		AC:0.993
Epoch 3250 (0.448 sec): loss changed from 0.14 to 0.126		VL:0.140		AC:0.996
Epoch 3260 (0.471 sec): loss changed from 0.132 to 0.124		VL:0.139		AC:0.994
Epoch 3270 (0.427 sec): loss changed from 0.136 to 0.124		VL:0.142		AC:0.993
Epoch 3280 (0.496 sec): loss changed from 0.134 to 0.121		VL:0.141		AC:0.990
Epoch 3290 (0.429 sec): loss changed from 0.138 to 0.126		VL:0.136		AC:0.996
Epoch 3300 (0.509 sec): loss changed from 0.143 to 0.123		VL:0.143		AC:0.993
Epoch 3310 (0.47 sec): loss changed from 0.142 to 0.129		VL:0.139		AC:0.994
Epoch 3320 (0.422 sec): loss changed from 0.14 to 0.125		VL:0.139		AC:0.993
Epoch 3330 (0.443 sec): loss changed from 0.132 to 0.12		VL:0.144		AC:0.993
Epoch 3340 (0.463 sec): loss changed from 0.141 to 0.131		VL:0.139		AC:0.994
Epoch 3350 (0.455 sec): loss changed from 0.131 to 0.121		VL:0.136		AC:0.995
Epo

In [34]:
#basic-rnn-5     1.140 loss, 0.303 accuracy (stopped at 200 epochs)
#gru-rnn-5       0.998 loss, 0.378 accuracy
#gru-rnn-10      0.524 loss, 0.681 accuracy (and still improving fast) -> 0.136 loss and 0.995 accuracy
#gru-rnn-5-5     0.590 loss, 0.664 accuracy (and still improving fast)
valid_y[:10], valid_r[:10,0]

(array([3, 4, 2, 2, 1, 2, 2, 2, 4, 3]),
 array([3, 3, 2, 3, 2, 2, 3, 2, 3, 3], dtype=int64))

In [21]:
#gru-rnn, 5, elu => 0.143 loss, 0.994 accuracy
valid_y[:10], valid_r[:10,0]

AttributeError: 'str' object has no attribute 'reversed'

In [16]:
time.time()

1506627282.2685657